In [35]:
import tweepy
from GetOldTweets import got

consumer_keys = ["51MI8RrYmzO4btCKG4Qb5uqAa", "glUJXqMWks3Hlu1QgPa2KvMHA"]
consumer_secrets = ["ajpPv3Ag0NvMEQLBIwiPyDyU78BbLZn8IS1gTba4x9ZOHNPMNM", "lDjQ3F9OraljWas6koMqYKzP9fFu6wdNiBEoBbvQUkNmr304Q1"]
access_tokens = ["3004471069-VDbNpT9NO0QOtiqKZXkoH5Flv4MArCflIYImXjn", "905545734748905472-1427NPHJ3VOgSxhygcK6gVNFTZgu4AE"]
access_tokens_secret = ["sP6KMjPZXxYAnaae8bOiauLjCVnx8bzWkBk4KU1iZBxdl", "vFrkF37p0B4C4cYiwMWhAHCiKj2r0wCrKcfutaWzg79fB"]

consumer_key = consumer_keys[1]
consumer_secret = consumer_secrets[1]
access_token = access_tokens[1]
access_token_secret = access_tokens_secret[1]

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 



In [36]:
def printTweets(tweets):
    print("# tweets is: "+str(len(tweets)))
    counter = 0
    for tweet in tweets:
        print(str(counter)+": " + tweet.text)


        counter += 1

In [37]:
def printDateText(tweets):
    print("# tweets is: "+str(len(tweets)))
    counter = 0
    for tweet in tweets:
        print(str(counter)+": " + str(tweet.date)+"  ||  " + tweet.text)



        counter += 1

In [38]:
def getSortCriteria(tweet):
    print(tweet)
    return tweet.date



In [39]:
def writeToFile(path, tweets, language=None):
    output = open("tweettext.txt", "wb")
    counter = 0

    for tweet in new_tweets[:n]:
        # printing the text stored inside the tweet object
    #    print(tweet.user.screen_name,"Tweeted:",tweet.text)
        if (language == None) or (language == tweet.lang):
            adjusted_tweet = (tweet.text).encode('utf-8')

            output.write(adjusted_tweet)
            print(str(counter)+adjusted_tweet)
            output.write("\n")

            print(tweet.created_at)

            counter += 1
            
  
    output.close()

In [40]:
import urllib2
from bs4 import BeautifulSoup as bs

#https://finance.yahoo.com/quote/AMZN
    
def get_historical_data(name, number_of_days):
    data = []
    url = "https://finance.yahoo.com/quote/" + name + "/history?period1=946713600&period2=1510732800&interval=1d&filter=history&frequency=1d/"
    rows = bs(urllib2.urlopen(url).read()).findAll('table')[0].tbody.findAll('tr')

    for each_row in rows:
        divs = each_row.findAll('td')
        if divs[1].span.text  != 'Dividend': #Ignore this row in the table
            #I'm only interested in 'Open' price; For other values, play with divs[1 - 5]
            data.append({'Date': divs[0].span.text, 'Open': float(divs[1].span.text.replace(',',''))})

    return data[number_of_days:number_of_days+1]


In [41]:
import csv


def getPriceFromCSV(filepath, date):
    with open(filepath, 'rt') as f:
        #skip header
        next(f)
        
        reader = csv.reader(f)
        rownum = 1
        for row in reader: 
            rownum += 1

            
            #CSV appears to be in mm/dd/yyyy format but parsing is yyyy/mm/dd ? weird
            adjusted_date = (dt.datetime.strptime(str(row[0]), '%Y-%m-%d')).date()

            if adjusted_date == date:
                print("price of interest is: " + str(row[1]))
                return str(row[1])
    

In [42]:
#returns filtered tweets
def filterTweets(tweets, remove_duplicates=True):
    combined_tweets = tweets
    if remove_duplicates: 
        combined_tweets = list(set(combined_tweets)) #remove duplicates
        
    counter = 0
    for tweet in combined_tweets:
    #    print(counter)
        try:
            if tweet.date:
                counter += 1
                continue
        except AttributeError:
            print("exception on number: " + str(counter))
            print(tweet.text)
           # combined_tweets.remove(tweet)
            combined_tweets = list(filter(lambda twt: twt != tweet, combined_tweets))

            counter += 1

    print("filtered: " + str(counter) )      
#    printDateText(combined_tweets)
    return combined_tweets

In [45]:
from yahoo_finance import Share
import datetime as dt
from datetime import timedelta
import os

remove_duplicates = True
query = "Amazon"
stock_ticker = "AMZN"
max_tweets = 55
n = max_tweets

#twitter was founded in 2006. Start in 2007 for safety
base_date = "2007-01-03"
start_date = "2007-01-04"
end_date = "2007-02-05"

#new_tweets = [tweet for tweet in tweepy.Cursor(api.search, 
                                   #        q=query).items(max_tweets)]

print("Pulling tweets...")    
#get old tweets

base_date_dt = dt.datetime.strptime(base_date, '%Y-%m-%d')
start_date_dt = dt.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = dt.datetime.strptime(end_date, '%Y-%m-%d')
delta = end_date_dt - start_date_dt

num_days = int(delta.days)
print(num_days)
begin = base_date_dt.strftime('%Y-%m-%d')

i = 1
tweets = []
processed_tweets = []

#perform data structure swapping and clearing to save memory for large runs
for single_date in (start_date_dt + timedelta(n) for n in range(num_days-1)):
    date_only = single_date.strftime('%Y-%m-%d')
    
    end = single_date.strftime('%Y-%m-%d')

    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query).setSince(begin).setUntil(end).setMaxTweets(max_tweets)
    tweets.extend(got.manager.TweetManager.getTweets(tweetCriteria))#[0]
    
    
    #filter the sets of tweets in batches of 20
    if ((i % 20) == 0) or ((num_days - i) < 20):
        processed_tweets.extend(filterTweets(tweets, remove_duplicates=True))
        tweets = []
        
        #sort by date
        processed_tweets.sort(key=getSortCriteria)
        #printDateText(combined_tweets)


        #find associated stock prices
        #stock = Share( stock_ticker )
        cur_date = dt.datetime.now()
        print(cur_date)

        tweet_labels = []
        cutoff_date = dt.datetime.strptime("2017-10-30", '%Y-%m-%d')


        datapath = os.getcwd()+"/StockData/AMZN.csv"
        cutoff_threshold = timedelta(days=0)

        for tweet in processed_tweets:
            tweet_date = tweet.date
            print("tweet date is: " + str(tweet_date) +" of type: " + str(type(tweet_date)))

            #pull from file since the tweet is too old
            if (tweet_date - cutoff_date) < cutoff_threshold:
                tweet_dateOnly = (tweet_date).date()
                open_price = getPriceFromCSV(datapath, tweet_dateOnly)
                if(open_price == None): #could have been a holiday
                    #processed_tweets.remove(tweet)
                    processed_tweets = list(filter(lambda twt: twt != tweet, processed_tweets))


                else:
                    tweet_labels.append(str(open_price))
        #            print("type is: " + str(type(open_price)))

            else:
                delta = cur_date - tweet_date
                delta_days = delta.days
                print("delta days: "+str(delta_days))

                price = get_historical_data(stock_ticker, delta_days)
                tweet_labels.extend(str(open_price))

        #test
        #print(get_historical_data(stock_ticker, 10))


        #print(counter)    
        #printDateText(processed_tweets)

        #dump to file
        path = os.getcwd()+"/"
        f = open(path+'tweets.txt', 'a')    
        for t in processed_tweets:
            f.write(((t.date).strftime('%Y-%m-%d')) + ",  " + (t.text).encode('utf-8'))
            f.write('\n')
        f.close()

        f2 = open(path+'labels.txt', 'a')
        with open(path+'labels.txt') as l:
            for price in tweet_labels:
                print(price)
                f2.write(price)
                f2.write('\n')
        f2.close()
        
        processed_tweets = []


#    printTweets(old_tweets)
    
    #set new begin to be the current day for the next iteration
    begin = end
    
    i+=1
    
      



lines_in_tweets = 0
f = open(path+'tweets.txt', 'r')
for line in f:
    lines_in_tweets += 1
    
f.close()

lines_in_labels = 0
f2 = open(path+'labels.txt', 'r')
for line in f2:
    lines_in_labels += 1
    
print("#lines text: " + str(lines_in_tweets) +"  #lines labels: " + str(lines_in_labels))


Pulling tweets...
32
filtered: 39
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e407830>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e43a050>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e51c878>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10db8b6c8>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e5832d8>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e562908>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e585128>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e648830>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e74f950>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e583440>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e74f9e0>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e419050>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e51ca28>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e4d8a70>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e57b878>
<GetOldTweets.got.models.Tweet.Tweet

filtered: 0
2017-11-30 00:01:57.303607
filtered: 0
2017-11-30 00:01:57.579393
filtered: 2
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e583440>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e4195f0>
2017-11-30 00:01:58.202328
tweet date is: 2007-01-20 05:45:09 of type: <type 'datetime.datetime'>
tweet date is: 2007-01-20 13:44:37 of type: <type 'datetime.datetime'>
filtered: 1
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e407cb0>
2017-11-30 00:01:58.934589
tweet date is: 2007-01-21 07:00:16 of type: <type 'datetime.datetime'>
filtered: 6
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e74f050>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e585cb0>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e74fd40>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e4df368>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10e73ad88>
<GetOldTweets.got.models.Tweet.Tweet instance at 0x10da87bd8>
2017-11-30 00:01:59.629485
tweet date is: 2007-01-21 17:40:31

price of interest is: 37.950001
tweet date is: 2007-02-01 09:36:13 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 09:38:45 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 09:39:44 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 09:44:51 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 10:26:34 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 13:05:08 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 13:34:37 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 13:59:10 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 14:00:21 of type: <type 'datetime.datetime'>
price of interest is: 37.950001
tweet date is: 2007-02-01 15:10:10 of typ